In [1]:
import numpy as np
import torch
import math 
from torch.utils.data import Dataset


def calculate_fire_spread_rate(wind_speed_at_10m, moisture_content, slope_angle, fuel_load):
    """
    Calculate the rate of spread of a forest fire.

    Parameters:
    wind_speed_at_10m (float): Wind speed at 10 meters height in km/hr.
    moisture_content (float): Moisture content in percentage (35% - 65%).
    slope_angle (float): Slope angle in degrees.
    fuel_load (float): Fuel load in tonnes/hectare (< 6 mm diameter), default is 2 tonnes/hectare.

    Returns:
    float: Rate of spread of the fire in unspecified units.
    """
    # Constants
    a = 1.674
    b = 0.1798
    c = 0.22
    d = 0.158
    e = -0.227
    f = 0.0662

    # Convert wind speed at 10m to wind speed at 1.5m
    wind_speed_at_1_5m = a + b * wind_speed_at_10m

    # Calculate the rate of spread on flat ground
    rate_flat_ground = c * fuel_load * math.exp((wind_speed_at_1_5m ** d) * (moisture_content ** e))

    # Adjust rate for slope
    rate_on_slope = rate_flat_ground * math.exp((slope_angle ** f))

    return rate_on_slope

# Example usage
spread_rate = calculate_fire_spread_rate(20, 50, 10, 2)
spread_rate

/Users/sadiahzahoor/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.4078009424508124

In [2]:
from scipy.stats import qmc
import pandas as pd


# Define the bounds for each parameter
bounds = np.array([[11.5, 34.1],  # Wind speed
                   [0, 6],        # Fuel load
                   [65, 100],      # Moisture content
                   [0, 30]])      # Slope angle

# Using Latin Hypercube Sampling (LHS) for a more structured approach
sampler = qmc.LatinHypercube(d=4)
sample = sampler.random(n=40000)
sample = qmc.scale(sample, bounds[:, 0], bounds[:, 1])

# Extracting the samples for each parameter
wind_speed_samples = sample[:, 0]
fuel_samples = sample[:, 1]
moisture_content_samples = sample[:, 2]
slope_angle_samples = sample[:, 3]

# Calculating spread rates for the sampled data
spread_rates = [calculate_fire_spread_rate(wind_speed, moisture, slope, fuel) 
                for wind_speed, moisture, slope, fuel in zip(wind_speed_samples, moisture_content_samples, slope_angle_samples, fuel_samples)]

# Creating a DataFrame to hold the sampled data and calculated spread rates
data_lhs = pd.DataFrame({
    'Wind Speed (km/hr)': wind_speed_samples,
    'Fuel Load (tonnes/hectare)': fuel_samples,
    'Moisture Content (%)': moisture_content_samples,
    'Slope Angle (degrees)': slope_angle_samples,
    'Spread Rate': spread_rates
})

data_lhs.head()

,Wind Speed (km/hr),Fuel Load (tonnes/hectare),Moisture Content (%),Slope Angle (degrees),Spread Rate
0,23.920019,1.141099,91.075957,7.429567,1.266346
1,17.514850,4.561891,65.257152,27.965450,5.737176
2,15.225474,4.178999,67.990689,24.427905,5.138176
3,33.717042,0.053136,87.637692,7.283106,0.060321
4,32.984927,0.726090,96.680123,13.922715,0.855888


In [3]:

import torch
from torch.utils.data import Dataset

class FireSpreadDataset(Dataset):
    def __init__(self, dataframe):
        # Convert the dataframe to a PyTorch tensor
        self.data = torch.tensor(dataframe.values, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Assuming the last column is the target variable (spread rate)
        features = self.data[idx, :-1]
        target = self.data[idx, -1]
        return features, target

# Convert the DataFrame to a PyTorch Dataset
fire_spread_data = FireSpreadDataset(data_lhs)

# Example: accessing the first item in the dataset
first_sample_features, first_sample_target = fire_spread_data[0]
print(first_sample_features, first_sample_target)


tensor([23.9200,  1.1411, 91.0760,  7.4296]) tensor(1.2663)


In [4]:
import numpy as np
import torch
from torch.utils.data import Dataset
from typing import List 
import torch.nn as nn
import torch.optim as optim

class BushfireModel(nn.Module): 
    def __init__(self, n: int, hidden_layers: List[int]): 
        super().__init__() #Inherit from the nn.module
        self.n = n # n input features
        self.hidden_layers = hidden_layers # hidden layers are the middle ones

        self.layers = nn.ModuleList([
            nn.Linear(in_dim, out_dim)
            for in_dim, out_dim in zip([n, *hidden_layers], [*hidden_layers, 1])
        ]) # Single output

    def forward(self, x: torch.tensor):
        for i, layer in enumerate(self.layers):
            x = layer(x)
            if i != len(self.layers) - 1:    #We don't want to ReLU the last layer
                x = nn.functional.relu(x)  
        return x # Linear output for regression

In [7]:
import torch
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim

# Splitting the dataset
total_size = len(fire_spread_data)
train_size = int(0.8 * total_size)  # 80% for training
val_size = total_size - train_size  # 20% for validation
train_dataset, val_dataset = random_split(fire_spread_data, [train_size, val_size])

# Creating DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Training function
def train_model(model, train_loader, val_loader, loss_criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()  # Training mode
        for i, (features, labels) in enumerate(train_loader):
            # Forward pass
            outputs = model(features)
            loss = loss_criterion(outputs, labels.unsqueeze(1))

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()  # Validation mode
        with torch.no_grad():
            val_loss = 0
            for features, labels in val_loader:
                outputs = model(features)
                val_loss += loss_criterion(outputs, labels.unsqueeze(1)).item()

        val_loss /= len(val_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}')

# Initialize the model
model = BushfireModel(n=4, hidden_layers=[200,100])
loss_criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Train the model
num_epochs = 50
train_model(model, train_loader, val_loader, loss_criterion, optimizer, num_epochs)


Epoch [1/50], Loss: 0.0832, Val Loss: 0.1069
Epoch [2/50], Loss: 0.0132, Val Loss: 0.0153
Epoch [3/50], Loss: 0.0095, Val Loss: 0.0092
Epoch [4/50], Loss: 0.0082, Val Loss: 0.0078
Epoch [5/50], Loss: 0.0031, Val Loss: 0.0044
Epoch [6/50], Loss: 0.0029, Val Loss: 0.0034
Epoch [7/50], Loss: 0.0106, Val Loss: 0.0040
Epoch [8/50], Loss: 0.0011, Val Loss: 0.0028
Epoch [9/50], Loss: 0.0023, Val Loss: 0.0036
Epoch [10/50], Loss: 0.0024, Val Loss: 0.0023
Epoch [11/50], Loss: 0.0105, Val Loss: 0.0023
Epoch [12/50], Loss: 0.0052, Val Loss: 0.0022
Epoch [13/50], Loss: 0.0025, Val Loss: 0.0019
Epoch [14/50], Loss: 0.0016, Val Loss: 0.0015
Epoch [15/50], Loss: 0.0011, Val Loss: 0.0014
Epoch [16/50], Loss: 0.0039, Val Loss: 0.0039
Epoch [17/50], Loss: 0.0010, Val Loss: 0.0013
Epoch [18/50], Loss: 0.0016, Val Loss: 0.0012
Epoch [19/50], Loss: 0.0011, Val Loss: 0.0029
Epoch [20/50], Loss: 0.0014, Val Loss: 0.0011
Epoch [21/50], Loss: 0.0014, Val Loss: 0.0014
Epoch [22/50], Loss: 0.0006, Val Loss: 0.00

In [8]:
def predict(model, input_features):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        predictions = model(input_features)
    return predictions

# Example usage with one sample from the validation set
features, labels = next(iter(val_loader))
prediction = predict(model, features)
print(f'Predicted: {prediction}, Actual: {labels}')


Predicted: tensor([[1.8862],
        [5.1778],
        [5.8263],
        [2.6930],
        [7.0239],
        [1.4026],
        [1.3653],
        [0.2444],
        [6.6313],
        [5.7894],
        [5.8786],
        [2.0052],
        [1.9386],
        [0.0687],
        [3.2190],
        [3.6501],
        [5.3795],
        [3.8314],
        [3.1406],
        [3.9680],
        [4.8607],
        [2.6079],
        [1.8070],
        [7.2219],
        [1.7048],
        [2.5089],
        [1.9950],
        [1.8502],
        [5.8966],
        [5.5424],
        [5.3046],
        [6.2302],
        [0.7887],
        [5.0433],
        [0.5690],
        [1.4547],
        [1.3146],
        [2.0013],
        [4.0048],
        [1.1692],
        [4.5764],
        [0.3091],
        [5.1381],
        [6.6209],
        [4.1047],
        [0.6963],
        [0.5893],
        [6.7630],
        [4.0035],
        [2.8910],
        [0.2033],
        [3.8691],
        [4.4518],
        [3.2199],
        [1.7774],